# 0. Load Data

In [1]:
import math 
import collections
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from joblib import dump

# Isolation Forest
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# VAE
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.layers import Input, Dense, Lambda, Flatten, Reshape
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras import backend as K
from keras import metrics

In [2]:
df = pd.read_csv("data/cleaned_KPIs.csv")
df.head()

,index,cell_id,DL_TRAFFIC_VOLUME,UL_TRAFFIC_VOLUME,Inter_X2_based_HO_prep,VoLTE_total_traffic,INTRA_FREQ_HO_SR_RATIO,RRC_SR_RATIO,CELL_AVAILABILITY_RATIO,RACH_Stp_Completion_SR_RATIO,...,DCR_LTE_RATIO,CSSR_LTE_RATIO,LTE_INTER_ENODEB_HOSR_RATIO,E_UTRAN_Inter_Freq_HO_SR_RATIO,Inter_RAT_HO_SR_GERAN_SRVCC_RATIO,Inter_RAT_Total_HO_SR_RATIO,E_UTRAN_tot_HO_SR_inter_eNB_X2_RATIO,E_RAB_DR_RATIO,HOUR,cell
0,2021-05-09 00:00:00,2.226537e+17,3.779737e+10,3.947172e+09,15.0,4727.0,0.809859,0.992427,1.0,0.962688,...,0.001761,0.996041,0.400000,0.770642,0.963636,0.963636,0.400000,0.001761,0,0
1,2021-05-09 01:00:00,2.226537e+17,3.684898e+10,4.088752e+09,6.0,3076.0,0.886792,0.993288,1.0,0.973207,...,0.002468,0.995465,0.500000,0.842105,1.000000,1.000000,0.500000,0.002468,1,0
2,2021-05-09 02:00:00,2.226537e+17,3.292677e+10,5.016897e+09,8.0,3501.0,0.938356,0.994664,1.0,0.966330,...,0.003077,0.996044,0.375000,0.931624,1.000000,0.966667,0.375000,0.003077,2,0
3,2021-05-09 03:00:00,2.226537e+17,3.021547e+10,5.139107e+09,9.0,2275.0,0.860215,0.994819,1.0,0.943216,...,0.001721,0.995920,0.777778,0.816901,0.947368,0.947368,0.777778,0.001721,3,0
4,2021-05-09 04:00:00,2.226537e+17,3.082176e+10,4.250716e+09,17.0,2178.0,0.840426,0.995952,1.0,0.936256,...,0.002213,0.995628,0.764706,0.794521,1.000000,1.000000,0.764706,0.002213,4,0


In [3]:
df = df.drop(['cell_id','index'],axis=1)

# 1. Isolation Forest

In [4]:
IF=IsolationForest(n_estimators=150, 
                      max_samples ='auto', 
                      max_features=1,random_state=42)

In [5]:
IF.fit(df)
# score_samples = - score  
IF_score = -1 * IF.score_samples(df)
IF_score[:20]

array([0.51295825, 0.48509757, 0.53648751, 0.50925568, 0.48977436,
       0.52157804, 0.50460632, 0.47403398, 0.47869537, 0.4786467 ,
       0.46769137, 0.46808774, 0.47676634, 0.48726571, 0.48067503,
       0.50214502, 0.47844321, 0.47284323, 0.46897991, 0.46007395])

# 2. VAE

[keras - Variational AutoEncoder 官方代码范例(复杂)](https://keras.io/examples/generative/vae/)

[keras - AE/VAE 相对简单的代码构建](https://blog.keras.io/building-autoencoders-in-keras.html)

[keras - 中等程度AVE 但是有点乱](https://github.com/keras-team/keras/blob/2c8d1d03599cc03243bce8f07ed9c4a3d5f384f9/examples/variational_autoencoder.py)







[中文VAE理论+代码](https://blog.csdn.net/weixin_37737254/article/details/102920263)

[keras - 函数式API](https://keras.io/guides/functional_api/)

[keras - Conv1D](https://keras-zh.readthedocs.io/layers/convolutional/)


## 2.1 Build model

**(1) Standardize** 

In [ ]:
df_scaled = StandardScaler().fit_transform(df)

**(2) Define sampling function**

In [7]:
def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),
                              mean=0., stddev=0.1)
    return z_mean + K.exp(z_log_sigma) * epsilon

**(3) Build Encoder**

In [8]:
original_dim = df.shape[1]
intermediate_dim = 32
latent_dim = 2

# encoder input
inputs = keras.Input(shape=(original_dim,))
x = layers.Dense(16, activation="relu")(inputs)
x = layers.Dense(4, activation="relu")(x)

# the output of encoder
z_mean      = layers.Dense(latent_dim, name="z_mean")(x)
z_log_sigma = layers.Dense(latent_dim, name="z_log_var")(x)
z           = layers.Lambda(sampling)([z_mean, z_log_sigma])

# Create encoder
encoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 20)]         0           []                               
                                                                                                  
 dense (Dense)                  (None, 16)           336         ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 4)            68          ['dense[0][0]']                  
                                                                                                  
 z_mean (Dense)                 (None, 2)            10          ['dense_1[0][0]']                
                                                                                            

**(4) Build Decoder**

In [9]:
# Create decoder
latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
x = layers.Dense(4, activation='relu')(latent_inputs)
x = layers.Dense(16,activation='relu')(x)
outputs = layers.Dense(original_dim, activation='sigmoid')(x)
decoder = keras.Model(latent_inputs, outputs, name='decoder')
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 z_sampling (InputLayer)     [(None, 2)]               0         
                                                                 
 dense_2 (Dense)             (None, 4)                 12        
                                                                 
 dense_3 (Dense)             (None, 16)                80        
                                                                 
 dense_4 (Dense)             (None, 20)                340       
                                                                 
Total params: 432
Trainable params: 432
Non-trainable params: 0
_________________________________________________________________


**(5) Build Variational AutoEncoder**

In [10]:
# Instantiate VAE model
outputs = decoder(encoder(inputs)[2])
vae = keras.Model(inputs, outputs, name='vae_mlp')

# Define Loss Function
reconstruction_loss = keras.losses.mean_squared_error(inputs, outputs)
kl_loss = -0.5 * K.sum(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma), axis=-1)
vae_loss = K.mean(reconstruction_loss + 0.01* kl_loss) 

# Optimiser 
adam = keras.optimizers.Adam(learning_rate=0.001)
# Add loss function
vae.add_loss(vae_loss)
vae.compile(optimizer=adam)


Notices:
- weight of kl_loss?  [theoretical insight](https://stats.stackexchange.com/questions/332179/how-to-weight-kld-loss-vs-reconstruction-loss-in-variational-auto-encoder) + [a more specific example](https://stats.stackexchange.com/questions/341954/balancing-reconstruction-vs-kl-loss-variational-autoencoder)
- reconsctruction_loss : if use `binary_crossentropy`( $BCE(y,p)=-y.log(p)-(1-y).log(1-p)$ ), the input value should be in 0~1. In our case, since we use 'StandardScaler', we'd better use `mse`, otherwise, when training the model, the loss becomes -inf 

**(6) Train model**

In [11]:
epochs = 500
batch_size = 128
callback = EarlyStopping(monitor='val_loss',patience=10)

# Train model
vae.fit(df_scaled, df_scaled,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(df_scaled, df_scaled),
        shuffle=True,
#         verbose=False,
        callbacks=[callback]
       )

Epoch 1/500
65/65 [==============================] - 1s 5ms/step - loss: 1.2260 - val_loss: 1.1705
Epoch 2/500
65/65 [==============================] - 0s 3ms/step - loss: 1.0934 - val_loss: 1.0361
Epoch 3/500
65/65 [==============================] - 0s 3ms/step - loss: 1.0082 - val_loss: 0.9867
Epoch 4/500
65/65 [==============================] - 0s 3ms/step - loss: 0.9718 - val_loss: 0.9576
Epoch 5/500
65/65 [==============================] - 0s 3ms/step - loss: 0.9474 - val_loss: 0.9382
Epoch 6/500
65/65 [==============================] - 0s 3ms/step - loss: 0.9314 - val_loss: 0.9247
Epoch 7/500
65/65 [==============================] - 0s 3ms/step - loss: 0.9173 - val_loss: 0.9107
Epoch 8/500
65/65 [==============================] - 0s 3ms/step - loss: 0.9011 - val_loss: 0.8915
Epoch 9/500
65/65 [==============================] - 0s 3ms/step - loss: 0.8834 - val_loss: 0.8761
Epoch 10/500
65/65 [==============================] - 0s 3ms/step - loss: 0.8707 - val_loss: 0.8645
Epoch 11/

65/65 [==============================] - 0s 3ms/step - loss: 0.7749 - val_loss: 0.7755
Epoch 84/500
65/65 [==============================] - 0s 2ms/step - loss: 0.7775 - val_loss: 0.7753
Epoch 85/500
65/65 [==============================] - 0s 3ms/step - loss: 0.7752 - val_loss: 0.7753
Epoch 86/500
65/65 [==============================] - 0s 3ms/step - loss: 0.7776 - val_loss: 0.7752
Epoch 87/500
65/65 [==============================] - 0s 2ms/step - loss: 0.7774 - val_loss: 0.7752
Epoch 88/500
65/65 [==============================] - 0s 3ms/step - loss: 0.7779 - val_loss: 0.7748
Epoch 89/500
65/65 [==============================] - 0s 2ms/step - loss: 0.7765 - val_loss: 0.7735
Epoch 90/500
65/65 [==============================] - 0s 2ms/step - loss: 0.7717 - val_loss: 0.7713
Epoch 91/500
65/65 [==============================] - 0s 3ms/step - loss: 0.7717 - val_loss: 0.7713
Epoch 92/500
65/65 [==============================] - 0s 3ms/step - loss: 0.7710 - val_loss: 0.7713
Epoch 93/500


65/65 [==============================] - 0s 3ms/step - loss: 0.7679 - val_loss: 0.7676
Epoch 165/500
65/65 [==============================] - 0s 2ms/step - loss: 0.7679 - val_loss: 0.7678
Epoch 166/500
65/65 [==============================] - 0s 3ms/step - loss: 0.7685 - val_loss: 0.7680
Epoch 167/500
65/65 [==============================] - 0s 3ms/step - loss: 0.7686 - val_loss: 0.7674
Epoch 168/500
65/65 [==============================] - 0s 2ms/step - loss: 0.7678 - val_loss: 0.7677
Epoch 169/500
65/65 [==============================] - 0s 3ms/step - loss: 0.7676 - val_loss: 0.7674
Epoch 170/500
65/65 [==============================] - 0s 3ms/step - loss: 0.7675 - val_loss: 0.7679
Epoch 171/500
65/65 [==============================] - 0s 3ms/step - loss: 0.7700 - val_loss: 0.7675
Epoch 172/500
65/65 [==============================] - 0s 3ms/step - loss: 0.7667 - val_loss: 0.7674
Epoch 173/500
65/65 [==============================] - 0s 3ms/step - loss: 0.7697 - val_loss: 0.7674
Epoc

65/65 [==============================] - 0s 4ms/step - loss: 0.7553 - val_loss: 0.7551
Epoch 246/500
65/65 [==============================] - 0s 3ms/step - loss: 0.7560 - val_loss: 0.7551
Epoch 247/500
65/65 [==============================] - 0s 3ms/step - loss: 0.7569 - val_loss: 0.7553
Epoch 248/500
65/65 [==============================] - 0s 3ms/step - loss: 0.7554 - val_loss: 0.7555
Epoch 249/500
65/65 [==============================] - 0s 3ms/step - loss: 0.7561 - val_loss: 0.7554
Epoch 250/500
65/65 [==============================] - 0s 3ms/step - loss: 0.7563 - val_loss: 0.7551
Epoch 251/500
65/65 [==============================] - 0s 3ms/step - loss: 0.7548 - val_loss: 0.7551
Epoch 252/500
65/65 [==============================] - 0s 4ms/step - loss: 0.7553 - val_loss: 0.7551
Epoch 253/500
65/65 [==============================] - 0s 3ms/step - loss: 0.7545 - val_loss: 0.7549
Epoch 254/500
65/65 [==============================] - 0s 3ms/step - loss: 0.7550 - val_loss: 0.7549
Epoc

65/65 [==============================] - 0s 3ms/step - loss: 0.7543 - val_loss: 0.7531
Epoch 327/500
65/65 [==============================] - 0s 3ms/step - loss: 0.7553 - val_loss: 0.7536
Epoch 328/500
65/65 [==============================] - 0s 3ms/step - loss: 0.7543 - val_loss: 0.7536
Epoch 329/500
65/65 [==============================] - 0s 3ms/step - loss: 0.7538 - val_loss: 0.7533
Epoch 330/500
65/65 [==============================] - 0s 3ms/step - loss: 0.7526 - val_loss: 0.7534
Epoch 331/500
65/65 [==============================] - 0s 3ms/step - loss: 0.7549 - val_loss: 0.7533
Epoch 332/500
65/65 [==============================] - 0s 3ms/step - loss: 0.7539 - val_loss: 0.7530
Epoch 333/500
65/65 [==============================] - 0s 3ms/step - loss: 0.7534 - val_loss: 0.7533
Epoch 334/500
65/65 [==============================] - 0s 3ms/step - loss: 0.7534 - val_loss: 0.7537
Epoch 335/500
65/65 [==============================] - 0s 3ms/step - loss: 0.7542 - val_loss: 0.7536
Epoc

65/65 [==============================] - 0s 3ms/step - loss: 0.7527 - val_loss: 0.7522
Epoch 408/500
65/65 [==============================] - 0s 3ms/step - loss: 0.7515 - val_loss: 0.7517
Epoch 409/500
65/65 [==============================] - 0s 3ms/step - loss: 0.7521 - val_loss: 0.7518
Epoch 410/500
65/65 [==============================] - 0s 3ms/step - loss: 0.7522 - val_loss: 0.7518
Epoch 411/500
65/65 [==============================] - 0s 3ms/step - loss: 0.7515 - val_loss: 0.7520
Epoch 412/500
65/65 [==============================] - 0s 3ms/step - loss: 0.7518 - val_loss: 0.7520
Epoch 413/500
65/65 [==============================] - 0s 3ms/step - loss: 0.7519 - val_loss: 0.7516
Epoch 414/500
65/65 [==============================] - 0s 3ms/step - loss: 0.7525 - val_loss: 0.7518
Epoch 415/500
65/65 [==============================] - 0s 3ms/step - loss: 0.7519 - val_loss: 0.7520
Epoch 416/500
65/65 [==============================] - 0s 3ms/step - loss: 0.7511 - val_loss: 0.7519
Epoc

## 2.2 [Detect Anomaly](https://towardsdatascience.com/hands-on-anomaly-detection-with-variational-autoencoders-d4044672acd5)
Detect anomaly by finding those who have a high reconstruction loss.


Steps:
1. Measure error between the original train (clean/normal) set and the output of the model, and generate an error vector representing the error term of each sample.

2. Find a relatively extreme value on that vector to use as your error threshold.

3. Run the model over the test or real data, in which anomalies are probably mixed with normal data.

4. Measure the reconstruction error and mark samples that exhibit an error term higher than the error threshold as anomalies.

In [19]:
def mse(original,reconstructed):
    return np.mean((original-reconstructed)**2,axis=1)

In [20]:
from sklearn.preprocessing import MinMaxScaler
pred  = vae.predict(df_scaled)

VAE_loss = mse(df_scaled,pred)
VAE_loss[:20]

array([0.9548081 , 0.73358319, 1.40899536, 0.40172662, 0.39875465,
       1.22539572, 4.24734052, 1.04168809, 1.48420087, 1.76684453,
       0.91322416, 0.37648604, 0.73476575, 1.40290654, 0.94696274,
       1.27051636, 1.24073557, 0.48180151, 0.81719654, 0.78864816])

# 3. VAE + IF

In [22]:
encoder_output = encoder.predict(df_scaled)
encoder_output

[array([[-0.49713367,  0.6343895 ],
        [-0.47102445,  0.651857  ],
        [-0.8752914 ,  0.7799051 ],
        ...,
        [ 0.8411997 , -0.09197786],
        [ 0.8411997 , -0.09197786],
        [ 0.92201024, -0.00449666]], dtype=float32),
 array([[-0.42058647, -1.0437922 ],
        [-0.43079686, -1.0465313 ],
        [-0.20457768, -1.0476806 ],
        ...,
        [-1.1643947 , -0.62823445],
        [-1.1643947 , -0.62823445],
        [-1.1942037 , -0.6319522 ]], dtype=float32),
 array([[-0.4847255 ,  0.61144465],
        [-0.45416445,  0.7111582 ],
        [-0.8980478 ,  0.6922674 ],
        ...,
        [ 0.8187069 , -0.0659976 ],
        [ 0.8264665 , -0.05013945],
        [ 0.9322601 ,  0.14467111]], dtype=float32)]

In [23]:
def feature_from_encoder(encoder_output):
    # take mean and log_var as features
    df_encoded_mean   = pd.DataFrame(encoder_output[0],columns=["mean1","mean2"])
    df_encoded_logvar = pd.DataFrame(encoder_output[1],columns=["log_var1","log_var2"])
    df_encoded        = df_encoded_mean.join(df_encoded_logvar)
    return df_encoded

df_encoded = feature_from_encoder(encoder_output)
df_encoded

,mean1,mean2,log_var1,log_var2
0,-0.497134,0.634390,-0.420586,-1.043792
1,-0.471024,0.651857,-0.430797,-1.046531
2,-0.875291,0.779905,-0.204578,-1.047681
3,-0.518749,0.620647,-0.412036,-1.041771
4,-0.481611,0.644075,-0.426752,-1.045181
...,...,...,...,...
8274,0.841200,-0.091978,-1.164395,-0.628234
8275,0.841200,-0.091978,-1.164395,-0.628234
8276,0.841200,-0.091978,-1.164395,-0.628234
8277,0.841200,-0.091978,-1.164395,-0.628234


In [24]:
VAE_IF=IsolationForest(n_estimators=150, 
                       max_samples ='auto', 
                       max_features=1,random_state=42)
VAE_IF.fit(df_encoded)

IsolationForest(max_features=1, n_estimators=150, random_state=42)

In [25]:
VAE_IF_score = -1 * VAE_IF.score_samples(df_encoded)
VAE_IF_score[:20]

array([0.48525011, 0.4847171 , 0.58558559, 0.48906119, 0.48701446,
       0.50837399, 0.65859537, 0.47622879, 0.48440507, 0.59527048,
       0.48428325, 0.47782173, 0.52540531, 0.54494306, 0.46734691,
       0.5168567 , 0.50935647, 0.51890465, 0.4884866 , 0.48272472])

# 4. Combination of results

In [26]:
df['IF_score'] = IF_score
df['IF_score'].describe()

count    8279.000000
mean        0.451213
std         0.030114
min         0.403969
25%         0.428277
50%         0.443766
75%         0.471093
max         0.599004
Name: IF_score, dtype: float64

In [27]:
df['VAE_loss'] = VAE_loss
df['VAE_loss'].describe()

count    8279.000000
mean        0.747234
std         2.107643
min         0.023347
25%         0.194180
50%         0.377740
75%         0.776217
max        97.686493
Name: VAE_loss, dtype: float64

Notice there is some extreme value in `VAE_loss`, which may affect our later visualization 

In [28]:
df['VAE_loss'].quantile(0.99)

4.974172584272193

In [29]:
df['VAE_IF_score'] = VAE_IF_score
df['VAE_IF_score'].describe()

count    8279.000000
mean        0.486291
std         0.039233
min         0.423997
25%         0.458176
50%         0.474988
75%         0.508637
max         0.715104
Name: VAE_IF_score, dtype: float64

In [30]:
df.head()

,DL_TRAFFIC_VOLUME,UL_TRAFFIC_VOLUME,Inter_X2_based_HO_prep,VoLTE_total_traffic,INTRA_FREQ_HO_SR_RATIO,RRC_SR_RATIO,CELL_AVAILABILITY_RATIO,RACH_Stp_Completion_SR_RATIO,Inter_RAT_HO_SR_UTRAN_SRVCC_RATIO,E_RAB_QCI1_DR_RATIO,...,E_UTRAN_Inter_Freq_HO_SR_RATIO,Inter_RAT_HO_SR_GERAN_SRVCC_RATIO,Inter_RAT_Total_HO_SR_RATIO,E_UTRAN_tot_HO_SR_inter_eNB_X2_RATIO,E_RAB_DR_RATIO,HOUR,cell,IF_score,VAE_loss,VAE_IF_score
0,3.779737e+10,3.947172e+09,15.0,4727.0,0.809859,0.992427,1.0,0.962688,0.576414,0.000000,...,0.770642,0.963636,0.963636,0.400000,0.001761,0,0,0.512958,0.954808,0.485250
1,3.684898e+10,4.088752e+09,6.0,3076.0,0.886792,0.993288,1.0,0.973207,0.576414,0.000000,...,0.842105,1.000000,1.000000,0.500000,0.002468,1,0,0.485098,0.733583,0.484717
2,3.292677e+10,5.016897e+09,8.0,3501.0,0.938356,0.994664,1.0,0.966330,0.000000,0.013889,...,0.931624,1.000000,0.966667,0.375000,0.003077,2,0,0.536488,1.408995,0.585586
3,3.021547e+10,5.139107e+09,9.0,2275.0,0.860215,0.994819,1.0,0.943216,0.576414,0.000000,...,0.816901,0.947368,0.947368,0.777778,0.001721,3,0,0.509256,0.401727,0.489061
4,3.082176e+10,4.250716e+09,17.0,2178.0,0.840426,0.995952,1.0,0.936256,0.576414,0.000000,...,0.794521,1.000000,1.000000,0.764706,0.002213,4,0,0.489774,0.398755,0.487014


# 5. Save Results

## 5.1 Save models

In [51]:
#save model
dump(IF,"model/IF.joblib")

['model/IF.joblib']

In [32]:
vae.save("model/VAE")
encoder.save("model/Encoder")
decoder.save("model/Decoder")

INFO:tensorflow:Assets written to: model/VAE\assets
INFO:tensorflow:Assets written to: model/Encoder\assets
INFO:tensorflow:Assets written to: model/Decoder\assets


In [33]:
#save model
dump(VAE_IF,"model/VAE_IF.joblib")

['model/VAE_IF.joblib']

## 5.2 Save Data

In [34]:
df.to_csv("data/KPIs_with_metrics.csv",index=False)